# Phase 2 Data Prep

## Environment setup

The code in section 1.1 is a repeat of the code in the first EDA notebook in order to bring in the data.

In [1]:
DATA_DIR = "../data/home-credit-default-risk"  # data folder will be present in the level as the code folder

In [2]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import os
import zipfile
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import warnings
import re
warnings.filterwarnings('ignore')

def load_data(in_path, name):
    df = pd.read_csv(in_path)
    print(f"{name}: shape is {df.shape}")
    print(df.info())
    display(df.head(5))
    return df

datasets={}  # data will be stored in a dictionary

## Data Load

In [3]:
%%time
ds_names = ("application_train", "application_test", "bureau","bureau_balance","credit_card_balance","installments_payments",
            "previous_application","POS_CASH_balance")

for ds_name in ds_names:
    datasets[ds_name] = load_data(os.path.join(DATA_DIR, f'{ds_name}.csv'), ds_name)

application_train: shape is (307511, 122)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB
None


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


application_test: shape is (48744, 121)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB
None


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


bureau: shape is (1716428, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB
None


...

previous_application: shape is (1670214, 37)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 1

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


POS_CASH_balance: shape is (10001358, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int64  
 1   SK_ID_CURR             int64  
 2   MONTHS_BALANCE         int64  
 3   CNT_INSTALMENT         float64
 4   CNT_INSTALMENT_FUTURE  float64
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int64  
 7   SK_DPD_DEF             int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB
None


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


CPU times: user 34.9 s, sys: 4.48 s, total: 39.3 s
Wall time: 56.1 s


In [4]:
for ds_name in datasets.keys():
    print(f'dataset {ds_name:24}: [ {datasets[ds_name].shape[0]:10,}, {datasets[ds_name].shape[1]}]')

dataset application_train       : [    307,511, 122]
dataset application_test        : [     48,744, 121]
dataset bureau                  : [  1,716,428, 17]
dataset bureau_balance          : [ 27,299,925, 3]
dataset credit_card_balance     : [  3,840,312, 23]
dataset installments_payments   : [ 13,605,401, 8]
dataset previous_application    : [  1,670,214, 37]
dataset POS_CASH_balance        : [ 10,001,358, 8]


In [5]:
appTrainDF = datasets["application_train"]
appTestDF = datasets["application_test"]
bureauDF = datasets["bureau"]
bureauBalDF = datasets["bureau_balance"]
creditCardBalDF = datasets["credit_card_balance"]
installmentPayDF = datasets["installments_payments"]
previousAppDF = datasets["previous_application"]
cashBalDF = datasets["POS_CASH_balance"]

In [6]:
# delete objects no longer needed for memory
import gc
gc.enable()
del datasets
gc.collect()

60

## Custom Transformers

### Feature Additions

In [7]:
# Create installment features
class InstallmentFeaturesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.l = []
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        # new feature creation from the Installment Payment file
        X['PAY_IS_LATE'] = X['DAYS_INSTALMENT'] - X['DAYS_ENTRY_PAYMENT']
        X['AMT_MISSED'] = X['AMT_INSTALMENT'] - X['AMT_PAYMENT']
        
        return X

In [8]:
# Create installment features
class CreditCardBalanceFeaturesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.l = []
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        # new feature creation from the Credit Card file
        X['DPD_MISSED'] = X['SK_DPD'] - X['SK_DPD_DEF']
        X['CREDIT_UTILIZED'] = X['AMT_CREDIT_LIMIT_ACTUAL'] - X['AMT_DRAWINGS_CURRENT']
        X['MIN_CREDIT_AMTMISS'] = X['AMT_INST_MIN_REGULARITY'] - X['AMT_PAYMENT_CURRENT']

        return X

In [9]:
# Create previous application features
class PreviousApplicationFeaturesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.l = []
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        # new feature in the Previous Application file
        X['INTEREST'] = X['CNT_PAYMENT'] * X['AMT_ANNUITY'] - X['AMT_CREDIT']
        X['INTEREST_PER_CREDIT'] = X['INTEREST'] / X['AMT_CREDIT']
        X['CREDIT_SUCCESS'] = X['AMT_APPLICATION'] - X['AMT_CREDIT']
        X['INTEREST_RT'] = 2 * 12 * X['INTEREST'] / (X['AMT_CREDIT'] * (X['CNT_PAYMENT'] + 1))
        
        return X

In [10]:
# Create application features
class ApplicationFeaturesAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.l = []
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        # credit to income ratio
        X['CREDIT_INCOME_RATIO'] = X['AMT_CREDIT'] / X['AMT_INCOME_TOTAL']
        
        # annuity to income ratio
        X['ANNUITY_INCOME_RATIO'] = X['AMT_ANNUITY'] / X['AMT_INCOME_TOTAL']
        
        # length of the credit term
        X['CREDIT_LENGTH'] = X['AMT_ANNUITY'] / X['AMT_CREDIT']
        
        # what is income to age ratio
        X['INCOME_AGE_RATIO'] = X['AMT_INCOME_TOTAL'] / X['DAYS_BIRTH']
        
        # what is credit to age ratio
        X['CREDIT_AGE_RATIO'] = X['AMT_CREDIT'] / X['DAYS_BIRTH']
        
        # what percent of applicants life have they been working at recent company
        X['DAYS_EMPLOYED_PERCENT'] = X['DAYS_EMPLOYED'] / X['DAYS_BIRTH']
        
        # add liability feature code
        conditions_temp = [
            (X['FLAG_OWN_CAR'] == 'Y') & (X['FLAG_OWN_REALTY'] == 'Y'),
            (X['FLAG_OWN_CAR'] == 'N') & (X['FLAG_OWN_REALTY'] == 'Y'),
            (X['FLAG_OWN_CAR'] == 'Y') & (X['FLAG_OWN_REALTY'] == 'N'),
            (X['FLAG_OWN_CAR'] == 'N') & (X['FLAG_OWN_REALTY'] == 'N')]
        
        values_temp = ['0', '1', '2', '3']
        
        X['HAS_LIBAILITY'] = np.select(conditions_temp, values_temp)
        
        return X

### Feature Aggregation

In [11]:
# Create aggregate features
class FeaturesAggregater(BaseEstimator, TransformerMixin):
    def __init__(self, grouping_feature = None, previousDFcolumns = None, prefix = ''):
        
        self.prefix = prefix
        self.previousDFcolumns = previousDFcolumns
        self.grouping_feature = grouping_feature
        self.numeric_stats = ["min", "max", "mean", "count", "sum"]
        self.categorical_stats = ["mean", "count", "sum"]
        self.agg_op_features = {}
        self.agg_features_names = [self.grouping_feature]
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        numeric_cols = list(X.columns[X.columns.isin(self.previousDFcolumns)])
        numeric_cols = [num for num in numeric_cols if num not in ['SK_ID_CURR','SK_ID_PREV','SK_ID_BUREAU']]
        categorical_cols = list(X.columns[~X.columns.isin(self.previousDFcolumns)])
        
        for f in numeric_cols:
            self.agg_op_features[f] = self.numeric_stats
            self.agg_features_names = self.agg_features_names + [self.prefix + "_" + f + "_" + s for s in self.numeric_stats]
            
        for f in categorical_cols:
            self.agg_op_features[f] = self.categorical_stats
            self.agg_features_names = self.agg_features_names + [self.prefix + "_" + f + "_" + s for s in self.categorical_stats]
        
        result = X.groupby(self.grouping_feature).agg(self.agg_op_features)
        result.columns = result.columns.droplevel()
        result = result.reset_index(level=[self.grouping_feature])
        result.columns = self.agg_features_names
        return result

### One Hot Encoder

In [12]:
# Create aggregate features (via pipeline)
class getDummies(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None): # no *args or **kargs
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):      
        result = pd.get_dummies(X, columns = self.columns)
        return result

### Missing Data Removal

In [13]:
# Remove missing columns
class MissingFeatureRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold = .6):
        self.threshold = threshold
        
    def fit(self, X, y=None):
        
        # get the percent of missingness in features
        percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending = False)
        
        # turn into a data frame
        missing_application_train_data  = pd.DataFrame(percent, columns=['Percent'])
        
        # get the columns with missingness exceeding the threshold
        self.columns_to_drop = list(missing_application_train_data.index[missing_application_train_data['Percent'] > self.threshold])
        
        return self
    
    def transform(self, X, y=None):
        
        # drop the columns with missingness over the threshold
        X = X.drop(columns = self.columns_to_drop, axis=1)
        
        return X

### Collinear Feature Removal

In [14]:
# Remove features with high colli
class CollinearFeatureRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold = .9):
        self.threshold = threshold
        
    def fit(self, X, y=None):
        
        # get the correlation matrix for the entire dataset after one hot encoding features
        correlation_matrix = X.head(1000).corr().abs()
        
        # get only the lower portion of collinearity matrix
        lower = correlation_matrix.where(np.tril(np.ones(correlation_matrix.shape), k=-1).astype(np.bool))
        
        # get the fields with correlation above threshold
        self.columns_to_drop = [index for index in lower.index if any(lower.loc[index] > self.threshold)]
        
        return self
    
    def transform(self, X, y=None):
        
        # drop the columns with collinearity over the threshold
        X = X.drop(columns = self.columns_to_drop, axis=1)
        
        return X

# Overview

Throughout this document, the following occurs.
* The POS cash balance, installment payment, and credit card balance files get rolled up to the previous applications file joined by SK_ID_PREV. 
* The bureau balance file gets rolled up to the bureau file joined by SK_ID_BUREAU.
* The joined previous application data gets rolled up to the applications file
* The joined bureau data gets rolled up to the applications file

# Previous Applications Workflow

* Step 1: New features from the EDA are added to POS cash balance, installment payment, and credit card
* Step 2: OHE categorical features in POS cash balance, installment payment, and credit card balance data sets for categorical features
* Step 3: Aggregate numeric features by calculating min, max, mean, count, and sum for each feature. Aggregate the categorical features by taking the mean, count, and sum.
* Step 4: After aggregating, remove columns that are identical to other columns
* Step 5: Join the balance data sets to the previous application data separately. Only select the SK_ID_PREV and SK_ID_CURR from the previous application data set before joining to the balance data sets separately. This avoids the additional fields from the previous application data being stored multiple times when joined individually to the balance data set.
* Step 6: For joined previous application datasets: Aggregate numeric features by calculating min, max, mean, count, and sum for each min, max, mean, count, and sum feature. Aggregate the categorical features by taking the mean, count, and sum for each mean, count, and sum feature created previously.
* Step 6.1: After aggregating, remove columns that are more than 60% missing with other columns. This is a great way to reduce the feature space prior to joining to application data. Remove collinear features that are more than 90% correlated (VIF>10 which is standard threshold). This is another great way to reduce the feature space before storing the file.
* Step 7: For previous application data: Aggregate numeric features by calculating min, max, mean, count, and sum for each feature. Aggregate the categorical features by taking the mean, count, and sum.
* Step 7.1: After aggregating, remove features that are more than 60% missing with other features. This is a great way to reduce the feature space prior to joining to application data.
* Step 8: Join the aggregated previous application data to the application data by SK_ID_CURR

## POS Cash Balance

In [15]:
# set pos cash pipeline
pos_pipe = Pipeline([
    ('ohe', getDummies()),
    ('aggregater', FeaturesAggregater(grouping_feature = 'SK_ID_PREV', previousDFcolumns = cashBalDF.columns, prefix = 'POS')),
    ('missing data remover', MissingFeatureRemover()),
    ('collineariy remover', CollinearFeatureRemover())
])

In [16]:
# aggregate the data
aggcashBalDF = pos_pipe.fit_transform(cashBalDF)

In [17]:
# delete objects no longer needed for memory
import gc
gc.enable()
del cashBalDF
gc.collect()

20

## Credit Card Balance

In [18]:
# get column names
creditCardBalDF_columns = list(creditCardBalDF.columns)
creditCardBalDF_columns.extend(['DPD_MISSED', 'CREDIT_UTILIZED', 'MIN_CREDIT_AMTMISS'])

# set credit card pipeline
credit_card_pipe = Pipeline([
    ('credit_card_features', CreditCardBalanceFeaturesAdder()),
    ('ohe', getDummies()),
    ('aggregater', FeaturesAggregater(grouping_feature = 'SK_ID_PREV', previousDFcolumns = creditCardBalDF_columns, prefix = 'CC')),
    ('missing data remover', MissingFeatureRemover()),
    ('collineariy remover', CollinearFeatureRemover())
])

In [19]:
# aggregate the data
aggcreditCardBalDF = credit_card_pipe.fit_transform(creditCardBalDF)

In [20]:
# delete objects no longer needed for memory
import gc
gc.enable()
del creditCardBalDF
gc.collect()

20

## Installment Payments

In [21]:
# get column names
installmentPayDF_columns = list(installmentPayDF.columns)
installmentPayDF_columns.extend(['PAY_IS_LATE', 'AMT_MISSED'])

# set installment pipeline
installment_pipe = Pipeline([
    ('installment_features', InstallmentFeaturesAdder()),
    ('ohe', getDummies()),
    ('aggregater', FeaturesAggregater(grouping_feature = 'SK_ID_PREV', previousDFcolumns = installmentPayDF_columns, prefix = 'INST')),
    ('missing data remover', MissingFeatureRemover()),
    ('collineariy remover', CollinearFeatureRemover())
])

In [22]:
# aggregate the data
agginstallmentPayDF = installment_pipe.fit_transform(installmentPayDF)

In [23]:
# delete objects no longer needed for memory
import gc
gc.enable()
del installmentPayDF
gc.collect()

20

## Previous Applications

### Merge in aggregate data sets

In [24]:
prev_apps_all_df = previousAppDF.merge(aggcashBalDF, how='left', on='SK_ID_PREV')
prev_apps_all_df = prev_apps_all_df.merge(agginstallmentPayDF, how='left', on='SK_ID_PREV')
prev_apps_all_df = prev_apps_all_df.merge(aggcreditCardBalDF, how='left', on='SK_ID_PREV')

In [25]:
#This should be true
len(prev_apps_all_df) == len(previousAppDF)

True

In [26]:
# delete objects no longer needed for memory
import gc
gc.enable()
del previousAppDF
gc.collect()

40

### Aggregate

In [27]:
# get column names
prev_apps_all_df_columns = list(prev_apps_all_df.columns)
prev_apps_all_df_columns.extend(['INTEREST', 'INTEREST_PER_CREDIT', 'CREDIT_SUCCESS', 'INTEREST_RT'])

# set previous app pipeline
previous_app_pipe = Pipeline([
    ('previous_application_features', PreviousApplicationFeaturesAdder()),
    ('ohe', getDummies()),
    ('aggregater', FeaturesAggregater(grouping_feature = 'SK_ID_CURR', previousDFcolumns = prev_apps_all_df_columns, prefix = 'PREV')),
    ('missing data remover', MissingFeatureRemover()),
    ('collineariy remover', CollinearFeatureRemover())
])

In [28]:
# aggregate the data
aggprev_apps_all_df = previous_app_pipe.fit_transform(prev_apps_all_df)

In [29]:
# delete objects no longer needed for memory
import gc
gc.enable()
del prev_apps_all_df
gc.collect()

20

# Bureau Workflow

* Step 1: OHE categorical features in bureau balance data sets for categorical features
* Step 2: Aggregate numeric features by calculating min, max, mean, count, and sum for each feature. Aggregate the categorical features by taking the mean, count, and sum.
* Step 3: After aggregating, remove columns that are identical to other columns
* Step 4: Join the balance data sets to the bureau data separately. Only select the SK_ID_BUREAU and SK_ID_CURR from the previous application data set before joining to the balance data sets separately. This avoids the additional fields from the bureau data being stored multiple times when joined individually to the balance data set.
* Step 5: For joined bureau dataset: Aggregate numeric features by calculating min, max, mean, count, and sum for each min, max, mean, count, and sum feature. Aggregate the categorical features by taking the mean, count, and sum for each mean, count, and sum feature created previously.
* Step 5.1: After aggregating, remove columns that are more than 60% missing with other columns. This is a great way to reduce the feature space prior to joining to application data.
* Step 6: For previous application data: Aggregate numeric features by calculating min, max, mean, count, and sum for each feature. Aggregate the categorical features by taking the mean, count, and sum.
* Step 6.1: After aggregating, remove features that are more than 60% missing with other features. This is a great way to reduce the feature space prior to joining to application data. Remove collinear features that are more than 90% correlated (VIF>10 which is standard threshold). This is another great way to reduce the feature space before storing the file.
* Step 7: Join the aggregated bureau data to the application data by SK_ID_CURR

## Bureau Balance

In [30]:
# get column names
bureauBalDF_columns = list(bureauBalDF.columns)

# set bureau balance pipeline
bureau_balance_pipe = Pipeline([
    ('ohe', getDummies()),
    ('aggregater', FeaturesAggregater(grouping_feature = 'SK_ID_BUREAU', previousDFcolumns = bureauBalDF_columns, prefix = 'BURBAL')),
    ('missing data remover', MissingFeatureRemover()),
    ('collineariy remover', CollinearFeatureRemover())
])

In [31]:
# aggregate the data
aggbureauBalDF = bureau_balance_pipe.fit_transform(bureauBalDF)

In [32]:
# delete objects no longer needed for memory
import gc
gc.enable()
del bureauBalDF
gc.collect()

20

## Bureau

### Merge in aggregate data set

In [33]:
bureau_all_df = bureauDF.merge(aggbureauBalDF, how='left', on='SK_ID_BUREAU')

In [34]:
# This should be true
len(bureau_all_df) == len(bureauDF)

True

In [35]:
# delete objects no longer needed for memory
import gc
gc.enable()
del bureauDF
gc.collect()

60

### Aggregate

In [36]:
# get column names
bureau_all_df_columns = list(bureau_all_df.columns)

# set bureau pipeline
bureau_pipe = Pipeline([
    ('ohe', getDummies()),
    ('aggregater', FeaturesAggregater(grouping_feature = 'SK_ID_CURR', previousDFcolumns = bureau_all_df_columns, prefix = 'BUR')),
    ('missing data remover', MissingFeatureRemover()),
    ('collineariy remover', CollinearFeatureRemover())
])

In [37]:
# aggregate the data
aggbureau_all_df = bureau_pipe.fit_transform(bureau_all_df)

In [38]:
# delete objects no longer needed for memory
import gc
gc.enable()
del bureau_all_df
gc.collect()

20

# Applications Workflow

## Applications - Train

In [39]:
# merge the previous application and balance data to app data
appTrainDF = appTrainDF.merge(aggprev_apps_all_df, how='left', on='SK_ID_CURR')
appTrainDF = appTrainDF.merge(aggbureau_all_df, how='left', on='SK_ID_CURR')

In [40]:
# set application pipeline
app_pipe = Pipeline([
    ('app_features', ApplicationFeaturesAdder()),
    ('missing data remover', MissingFeatureRemover())
])

In [41]:
# transform the data
final_appTrainDF = app_pipe.fit_transform(appTrainDF)

In [ ]:
# print out final shape of data
final_appTrainDF.shape

In [43]:
# write out the new application training data
final_appTrainDF.to_csv(DATA_DIR + '/train_clean.csv')

## Applications - Test

In [44]:
# merge the previous application and balance data to app data
appTestDF = appTestDF.merge(aggprev_apps_all_df, how='left', on='SK_ID_CURR')
appTestDF = appTestDF.merge(aggbureau_all_df, how='left', on='SK_ID_CURR')

In [45]:
# transform the data
final_appTestDF = app_pipe.transform(appTestDF)

In [ ]:
# print out final shape of data
final_appTestDF.shape

In [47]:
# write out the new application test data
final_appTestDF.to_csv(DATA_DIR + '/test_clean.csv')